In [1]:
#Custom Functions and Imports
from helper import *

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


In [2]:
# Read csv files and clean ratings and date columns
beers = pd.read_csv('../data/beer_info.csv')
reviews = pd.read_csv('../data/beer_reviews.csv')
beers.set_index('beer_id',inplace=True)
reviews.set_index('review_id',inplace=True)
reviews.ratings = clean_ratings(reviews.ratings)
reviews.posted = clean_dates(reviews.posted)

In [ ]:
# Clean text data
rev_text = reviews.text
rev_text = rev_text.apply(lambda x: re.sub('[^\w\s]',' ',str(x).lower())).apply(lambda x: re.sub('\s+',' ',x))
rev_text = rev_text.apply(lambda x: x.strip())

In [7]:
reviews[['text']].to_csv()

pandas.core.frame.DataFrame

In [ ]:
reviews.groupby('username').agg({'username':'count'}).sort_values(by='username')

In [5]:
def user_item_review_matrix(beers_):
    '''Generates a user_item_review matrix provided a list of beers.'''
    '''Used in collaborative filtering'''
    beer_tdf = (
        beers[['beer_name','beer_style',]]
        [beers.index.isin(beers_)]
    )
    return (        
        reviews[reviews.beer_id.isin(beer_tdf.index)]
        .merge(beer_tdf,how='left',on='beer_id')
        [['username','score','beer_id','posted']]
        .drop_duplicates(subset=['beer_id','username'],keep='first')
        .pivot(index='username',columns='beer_id',values='score')
    )
from scipy.stats.stats import pearsonr
def corr(X, Y):
    r = np.corrcoef(X,Y)[0,1]
    f = 0.5 * np.log((1+r)/(1-r))
    se=1/np.sqrt(len(X)-3)
    ucl=f+2*se
    lcl=f-2*se
    lcl=(np.exp(2*lcl)-1) / (np.exp(2*lcl)+1)
    ucl=(np.exp(2*ucl)-1) / (np.exp(2*ucl)+1)
    return r,lcl,ucl

def generate_recommendations(list_,scores_):
    '''Takes a list of beer_ids and personal scores, and generates a list of beer_ids to try.
    Uses Pearson Coefficient Method '''
    # Create a DataFrame to store users products and ratings
    user_reviews = (
        pd.DataFrame({'username':'user','score':scores_,'beer_id':list_})
        .pivot(index='username',columns='beer_id',values='score')        
    )
    
    #Generate a User-Item-Review matrix
    uirm = user_item_review_matrix(list(user_reviews.columns))
    uirm_df = pd.concat([user_reviews, uirm])
    
    
    # Calculate the Pearson Correlation to user ratings
    rankings = {'user':[],'corr':[],'pairs':[]}    
    for i in range(1,len(uirm_df)):
        new_df = uirm_df.iloc[[0,i],:]
        new_df = new_df.dropna(axis=1, how='any')    
        if new_df.shape[1] >= 2:
            val = list(new_df.values)
            corr(val[0],val[1])[0]            
            rankings['user'].append(new_df.index[1])
            rankings['corr'].append(corr(val[0],val[1])[0])
            rankings['pairs'].append(new_df.shape[1])    
    ranking_df = pd.DataFrame(rankings)
    ranking_df.set_index('user',inplace=True)   
    
    recommender = (
        pd.merge(ranking_df,uirm,how='left',left_index=True,right_index=True)
        .dropna(axis=1,how='all')
    )

    similar_beers = (
        reviews[reviews['username'].isin(list(recommender.index))]
        .merge(beers,how='left',on='beer_id')
        [['username','score','beer_name','beer_id']]    
        .drop_duplicates(subset=['beer_id','username'],keep='first')
        .pivot(index='username',columns='beer_id',values='score')
    )
   
    try_these = pd.concat([recommender,similar_beers],axis=1, 
                          join_axes=[recommender.index]).drop(list(recommender.columns)[2:],axis=1)
   
    try_these.loc['total'] = try_these.sum(axis=0)
    try_these.loc['sim.sum'] = try_these.mul(try_these['corr'],axis=0).sum()    
    try_these.loc['total/sim.sum'] = try_these.loc['total'] / try_these.loc['sim.sum']     
    top_5 = list(try_these.T['total/sim.sum'].sort_values(ascending=False).head(5).index)
    return beers[beers.index.isin(top_5)][['beer_name','brewery','beer_style','BAscore']]

In [4]:


generate_recommendations([9653,9650,2678,9033,977,4146,982],[3.7,4.4,2.8,3.2,4.2,4.7,4.2])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in sqrt
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2400: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2401: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


,beer_name,brewery,beer_style,BAscore
beer_id,,,,
611,Solis Occasus - West Coaster IPA,Mike Hess Brewing - Miramar,American IPA,3.81
1840,Suparillo,Other Half Brewing Co.,American Imperial IPA,4.18
3793,Apiary,Kane Brewing Company,Belgian Saison,3.84
6294,Dark Secret,BrickStone Restaurant & Brewery,American Imperial Stout,4.32
7474,As You Wish...,Pipeworks Brewing Company,American Imperial Porter,4.43


In [ ]:
#beers[(beers.beer_name == 'Lager') & (beers.brewery == 'Brooklyn Brewery')]
#beers[(beers.beer_name == 'Two Hearted Ale')]


In [ ]:
select_random_user = reviews.sample(1)['username'].values
select_user_reviews = (
    reviews[reviews.username==select_random_user[0]]
    .sort_values(by=['beer_id','posted'])
    #.drop_duplicates(subset=["beer_id"], keep='first')
)
#select_user_reviews
user_df = (
    select_user_reviews
    .merge(beers,how='left',on='beer_id')
    [['username','score','beer_name','beer_id']]    
    .drop_duplicates(subset=['beer_id','username'],keep='first')
    .pivot(index='username',columns='beer_id',values='score')
)
user_df

In [ ]:
uirm = user_item_review_matrix(list(user_df.columns))

In [ ]:
uirm_df = pd.concat([user_df,uirm])
rankings = {'user':[],'corr':[],'pairs':[]}
for i in range(1,len(uirm_df)):
    new_df = uirm_df.iloc[[0,i],:]
    new_df = new_df.dropna(axis=1,how='any')     
    if new_df.shape[1] < 2:
        break
    val = list(new_df.values)
    corr(val[0],val[1])[0]
    rankings['user'].append(new_df.index[1])
    rankings['corr'].append(corr(val[0],val[1])[0])
    rankings['pairs'].append(new_df.shape[1])

In [ ]:
ranking_df = pd.DataFrame(rankings)
ranking_df.set_index('user',inplace=True)
recommender = (
    pd.merge(ranking_df,uirm,how='left',left_index=True,right_index=True)
    .dropna(axis=1,how='all')
)

for i in range(len(recommender)):
    total = recommender

#recommender.loc['total']=recommender.sum(axis=0,skipna=True)

[]

similar_beers = (
    reviews[reviews['username'].isin(list(recommender.index))]
    .merge(beers,how='left',on='beer_id')
    [['username','score','beer_name','beer_id']]    
    .drop_duplicates(subset=['beer_id','username'],keep='first')
    .pivot(index='username',columns='beer_id',values='score')
)
recommender

In [ ]:
try_these = pd.concat([recommender,similar_beers],axis=1, join_axes=[recommender.index]).drop(list(recommender.columns)[2:],axis=1)
try_these.loc['total'] = try_these.sum(axis=0)
try_these.loc['sim.sum'] = try_these.mul(try_these['corr'],axis=0).sum()    
try_these.loc['total/sim.sum'] = try_these.loc['total'] / try_these.loc['sim.sum']
top_5 = list(try_these.T['total/sim.sum'].sort_values(ascending=False).head(5).index)
beers[beers.index.isin(top_5)][['beer_name','brewery','beer_style','BAscore']]

In [ ]:
uirm_df = pd.concat([user_df,uirm])
for i in range(1,len(uirm_df)):
    new_df = uirm_df.iloc[[0,i],:]
    new_df = new_df.dropna(axis=1,how='any')
    if len(new_df) > 5:
        print(new_df)

In [ ]:
uirm

In [ ]:
uirm_df = pd.concat([user_df,uirm])

In [ ]:
reviews_df = pd.DataFrame({'id': pd.Series(reviews.beer_id),
                           'text':pd.Series(rev_text),
                           'score':pd.Series(reviews.score)})
reviews_df.head(4)

## Testing NLP on IPA Beers

In [ ]:
len(list(beers[beers.beer_style=='Leipzig Gose'].index))

In [ ]:
cat_ipa = list(beers[beers.beer_style=='American IPA'].index)
ipa_revs = reviews_df[reviews_df.id.isin(cat_ipa)]

cat_lzg = list(beers[beers.beer_style=='Leipzig Gose'].index)
lza_revs = reviews_df[reviews_df.id.isin(cat_lzg)]


#ipa_revs = testing[testing.id.isin(cat_ipa)]
#ipa_revs['score'] = list(map(int,list(map(round,ipa_revs.score))))

In [ ]:
freq_words(lza_revs.text, 20)

In [ ]:
lza_revs.text = make_corpus(lza_revs)

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=reviews_2, start=2, limit=40, step=6)

In [ ]:
lza_revs.text

In [ ]:
lda_model.print_topics()

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

In [ ]:
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus_df).toarray()
y = ipa_revs.score.values
X_train, X_text, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [ ]:
#model = RandomForestClassifier(n_estimators = 501, criterion = 'entropy')
#model.fit(X_train, y_train)

In [ ]:
ipa_revs = testing[testing.id.isin(cat_ipa)]
ipa_revs['score'] = list(map(int,list(map(round,ipa_revs.score))))

In [ ]:
%reset